In [52]:
using MAT
using DynamicPolynomials, LinearAlgebra, SparseArrays

data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
D = matread(data*"/WeightsDigits4.mat");
W1 = D["W1"]; W2 = D["W2"]; c = D["c"]; x_bar=D["x_bar"]; y_bar=D["y_bar"]
eps = 0.1;

m1=size(W1,2)
m2=size(W2,2)
m3=size(W2,1)

50

In [53]:
y_bar+1

5

In [54]:
size(c)

(10, 50)

In [55]:
@polyvar x1[1:m1] x2[1:m2] x3[1:m3]# variables


f=(c[y_bar+1,:]-c[10,:])'*x3

g=Vector{Polynomial{true,Float64}}([])
h=Vector{Polynomial{true,Float64}}([])

pol=1.0*x1[1]

for j in 1:m2
    pol=x2[j]-sum(W1[j,r]*x1[r] for r=1:m1)
    append!(g,[pol])
    append!(h,[x2[j]*pol])
end
for j in 1:m3
    pol=x3[j]-sum(W2[j,r]*x2[r] for r=1:m2)
    append!(g,[pol])
    append!(h,[x3[j]*pol])
end

for t in 1:m1
    append!(g,[-x1[t]+x_bar[1,t]+eps])
end

m=length(g)
l=length(h)

f=f([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])

for j in 1:m
    g[j]=g[j]([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])
end

for j in 1:l
    h[j]=h[j]([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])
end


x=[x1;x2;x3]; n=length(x)
println("***Problem setting***")
println("Number of variables: n=",n)
println("Number of inequality constraints: m=",m)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=214
Number of inequality constraints: m=214
Number of equality constraints: l=150


In [56]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);


k=0
s=166
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))+1

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=0
Sparsity order: s=166
Sparsity order: d=2
------------------------------------------------------
The clique sizes of varibles:
[164, 101]
[1, 50]
------------------------------------------------------
  Number of cliques: p=51
  Largest clique size: u=164


InterruptException: InterruptException:

In [57]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl")

k=1
s=76
L=100*ones(Float64,74)

@time opt_val1,opt_sol=InterRelax.RelaxSparse_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,s,k,L=L,
    assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

LoadError: LoadError: InterruptException:
in expression starting at /home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/InterruptedRelax/src/SolveRelaxDense.jl:0

In [58]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);


k=1
s=76
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))+1

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

LoadError: LoadError: InterruptException:
in expression starting at /home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/InterruptedRelax/src/SolveRelaxDense_without_multiplier.jl:2

In [59]:
using SparseArrays, TSSOS

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=1

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS="MD",TS=false,solver="Mosek");

26  2.6e-10  9.4e-13  4.7e-12  -2.68e-01  -2.341557941e+04  -2.344136433e+04  9.3e-13  2073.83
Optimizer terminated. Time: 2095.50 

SDP solving time: 2096.521307138 seconds.
termination status: INTERRUPTED
solution status: UNKNOWN_RESULT_STATUS
optimum = -23415.579411484905
2100.727045 seconds (33.45 M allocations: 5.366 GiB, 0.03% gc time)


In [60]:
binomial(11+1,1)

12

In [61]:
using SparseArrays, TSSOS

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=2

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS="MD",TS=false,solver="Mosek");

LoadError: LoadError: InterruptException:
in expression starting at /home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/InterruptedRelax/src/SolveRelaxDense_without_multiplier.jl:0

In [62]:
binomial(74+2,2)

2850